In [ ]:
# With this operation, a comprehensive language model is downloaded from the 'spacy' library. There are three versions to it:
# - Large
# - Medium
# - Small
# In this case, we are using the smaller model, which is around 30 MB.

!python -m spacy download en_core_web_sm
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!pip install openai==0.27.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 32.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# drive.mount() loads the contents from your Google Drive.

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install gradio

In [ ]:
!pip install -U sentence-transformers rank_bm25

In [ ]:
import json
import pandas as pd
import time
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
import nltk
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
# import tiktoken
from openai.embeddings_utils import get_embedding, cosine_similarity

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Read the data for Paris Hotels

In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/hamzafarooq/maven-mlsystem-design-cohort-1/main/data/miami_hotels.csv')

In [ ]:
df.head()

,id,type,name,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,prices,latitude,longitude,webUrl,website,rankingString,rankingDenominator,numberOfReviews,review,title
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place


In [ ]:
df.shape

(2511, 27)

In [ ]:
# Count the number of times a specific name appears in the 'name' column, and relay that number for each different name, in order.
df.name.value_counts()

,count
name,
Faena Miami Beach,45
Setai Hotel Miami,45
Best Western Plus Atlantic Beach Resort,45
1 Hotel South Beach,45
Nobu Hotel Miami Beach,45
...,...
"The Julia Hotel, Miami Beach",6
Beach Park Hotel,4
Bikini Lodge,4


In [ ]:
# Drop the duplicated values from each column, i.e. drop a row if it contains a duplicated value.

df=df.drop_duplicates()

In [ ]:
df.shape

(2511, 27)

Create a new folder in Google Drive - called "Semantic_Search"

In [ ]:
df.name.value_counts()

,count
name,
Faena Miami Beach,45
Setai Hotel Miami,45
Best Western Plus Atlantic Beach Resort,45
1 Hotel South Beach,45
Nobu Hotel Miami Beach,45
...,...
"The Julia Hotel, Miami Beach",6
Beach Park Hotel,4
Bikini Lodge,4


In [ ]:
# Make a folder in your drive folder called "Semantic_Search".
!mkdir /content/drive/MyDrive/Semantic_Search

mkdir: cannot create directory ‘/content/drive/MyDrive/Semantic_Search’: File exists


In [ ]:
# Save the dataframe to the folder you just creagted.
df.to_csv('/content/drive/MyDrive/Semantic_Search/miami_hotels.csv',index=False)

In [ ]:
df.shape

(2511, 27)

In [ ]:
df.name.value_counts()

,count
name,
Faena Miami Beach,45
Setai Hotel Miami,45
Best Western Plus Atlantic Beach Resort,45
1 Hotel South Beach,45
Nobu Hotel Miami Beach,45
...,...
"The Julia Hotel, Miami Beach",6
Beach Park Hotel,4
Bikini Lodge,4


In [ ]:
df.head()

,id,type,name,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,prices,latitude,longitude,webUrl,website,rankingString,rankingDenominator,numberOfReviews,review,title
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,[],25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place


In [ ]:
# Create a column named 'combined', which containes the titles of the different lodges, with the descriptions associated to it.
df["combined"] = (
    "title: " + df.title.str.strip()+"; Content: " + df.review.str.strip()
    # +"; desc: "+ df.text.str.strip()
)

In [ ]:
df.head()

,id,type,name,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,latitude,longitude,webUrl,website,rankingString,rankingDenominator,numberOfReviews,review,title,combined
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,title: My favorite hotel in Miami--Petar is th...
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay,title: Exquisite Stay; Content: There are not ...
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,title: Burlesqu Show; Content: I went for the ...
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,title: Dana at Gitano is a rockstar!; Content:...
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place,title: Perfect Place; Content: I would like to...


In [ ]:
import re

df_combined = df.copy()

df_combined['combined'] = df_combined['combined'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]','',str(x)))

# Translate all the "combined" column to lower case.
def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df_combined['combined']= df_combined['combined'].apply(lambda x: lower_case(x))


In [ ]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch

embedder = SentenceTransformer('all-mpnet-base-v2')

# Use the GPU if available
if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")
else:
  print("GPU Found!")
  embedder =  embedder.to('cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

GPU Found!


In [ ]:
df_combined.head()

,id,type,name,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,latitude,longitude,webUrl,website,rankingString,rankingDenominator,numberOfReviews,review,title,combined
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,title my favorite hotel in miamipetar is the b...
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay,title exquisite stay content there are not eno...
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,title burlesqu show content i went for the bur...
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,title dana at gitano is a rockstar content dan...
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place,title perfect place content i would like to gi...


In [ ]:
# Take a sample of the first 10 rows.
sample = df_combined[:10]

In [ ]:
sample

,id,type,name,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,latitude,longitude,webUrl,website,rankingString,rankingDenominator,numberOfReviews,review,title,combined
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,title my favorite hotel in miamipetar is the b...
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay,title exquisite stay content there are not eno...
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,title burlesqu show content i went for the bur...
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,title dana at gitano is a rockstar content dan...
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place,title perfect place content i would like to gi...
5,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I have to create time to come write this revie...,Stay at Faena hotel,title stay at faena hotel content i have to cr...
6,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Faena is over-rated. \n\nIt would be a 3 star ...,Overrated. There are better luxury options on ...,title overrated there are better luxury option...
7,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,The service at the Faena is exceptional. From ...,5 star service ALL around,title 5 star service all around content the se...
8,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,My girlfriend and I had the dreamiest stay! Ev...,The perfect lover’s getaway,title the perfect lovers getaway content my gi...
9,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,25.807375,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels 

In [ ]:
# Switch to CPU
embedder =  embedder.to('cpu')

startTime = time.time()

# Create a column named 'embedding', where the 'combined' column is turned to embeddings by the model.
sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))


Execution time in seconds: 5.028539657592773


<ipython-input-25-8160b5948b46>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))


In [ ]:
# Switch once more to GPU.

embedder =  embedder.to('cuda')
startTime = time.time()

sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

# Notice the difference in time to do this operation between GPU and CPU.


Execution time in seconds: 1.821979284286499


<ipython-input-26-c2ada10f2885>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))


In [ ]:
sample

,id,type,name,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,longitude,webUrl,website,rankingString,rankingDenominator,numberOfReviews,review,title,combined,embedding
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,title my favorite hotel in miamipetar is the b...,"[-0.03406972, 0.00075161044, -0.006820292, -0...."
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay,title exquisite stay content there are not eno...,"[-0.003170278, 0.0046354914, -0.034291837, 0.0..."
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,title burlesqu show content i went for the bur...,"[-0.04177331, 0.04754442, -0.0032742221, 0.004..."
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,title dana at gitano is a rockstar content dan...,"[0.038456447, 0.073768765, -0.0072073163, 0.03..."
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place,title perfect place content i would like to gi...,"[-0.0047579487, 0.051271785, -0.047507547, 0.0..."
5,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I have to create time to come write this revie...,Stay at Faena hotel,title stay at faena hotel content i have to cr...,"[-0.015282154, 0.0018193707, -0.020907281, 0.0..."
6,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Faena is over-rated. \n\nIt would be a 3 star ...,Overrated. There are better luxury options on ...,title overrated there are better luxury option...,"[-0.023257997, -0.015723582, 0.0051887627, 0.0..."
7,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,The service at the Faena is exceptional. From ...,5 star service ALL around,title 5 star service all around content the se...,"[0.027946766, 0.01693837, -0.006997557, 0.0678..."
8,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,My girlfriend and I had the dreamiest stay! Ev

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')

In [ ]:
# Transform your dataframe to a pickle file, which is a byte stream file used to save a dataframe's state across sections.
sample.to_pickle('/content/drive/MyDrive/Semantic_Search/df.pkl')    #to save the dataframe, df to 123.pkl

In [ ]:
# Load the pickle file.
df_with_embedding = pd.read_pickle('/content/drive/MyDrive/Semantic_Search/df.pkl') #to load 123.pkl back to the dataframe df

In [ ]:
df_with_embedding.head()

,id,type,name,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,longitude,webUrl,website,rankingString,rankingDenominator,numberOfReviews,review,title,combined,embedding
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,title my favorite hotel in miamipetar is the b...,"[-0.03406972, 0.00075161044, -0.006820292, -0...."
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay,title exquisite stay content there are not eno...,"[-0.003170278, 0.0046354914, -0.034291837, 0.0..."
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,title burlesqu show content i went for the bur...,"[-0.04177331, 0.04754442, -0.0032742221, 0.004..."
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,title dana at gitano is a rockstar content dan...,"[0.038456447, 0.073768765, -0.0072073163, 0.03..."
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place,title perfect place content i would like to gi...,"[-0.0047579487, 0.051271785, -0.047507547, 0.0..."


In [ ]:
query = 'Not worth the effort or money + This hotel is not worth the effort or the price'

# Embed the previous query.
query_embedding = embedder.encode(query,show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# First element of the 'combined' column.
df_with_embedding.combined[0]

'title my favorite hotel in miamipetar is the best content hands down my absolute favorite hotel in south beachtheres no place id rather stay ive been back four times in the past six months each experience is better than the last and i always look forward to coming back as impressive as the hotel is the team of people are what make each stay exceptional everyone is so friendly and they go out of their way to ensure that i have a perfect stay every single time petar in particular always ensures that my stay is wonderful every time and gives me total peace of mind every time i stay there  i never have to worry about a thing and he goes above and beyond to provide excellent service\n\nive stayed at many hotels in south beachmiami and theyre in a league of their own'

In [ ]:
# Create a list of embeddings, from the contents of the "combined" column.
corpus_embeddings = embedder.encode(df_with_embedding.combined,show_progress_bar=True)

# Create a column called 'similarity', displaying the cosine similarity between your embedded query and each embedded content from the 'corpus_embeddings' variable.
df_with_embedding['similarity']=cosine_similarity(corpus_embeddings, query_embedding)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
df_with_embedding

,id,type,name,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,webUrl,website,rankingString,rankingDenominator,numberOfReviews,review,title,combined,embedding,similarity
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,title my favorite hotel in miamipetar is the b...,"[-0.03406972, 0.00075161044, -0.006820292, -0....",0.136912
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay,title exquisite stay content there are not eno...,"[-0.003170278, 0.0046354914, -0.034291837, 0.0...",0.162131
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,title burlesqu show content i went for the bur...,"[-0.04177331, 0.04754442, -0.0032742221, 0.004...",0.049958
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,title dana at gitano is a rockstar content dan...,"[0.038456447, 0.073768765, -0.0072073163, 0.03...",0.102215
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place,title perfect place content i would like to gi...,"[-0.0047579487, 0.051271785, -0.047507547, 0.0...",0.111666
5,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I have to create time to come write this revie...,Stay at Faena hotel,title stay at faena hotel content i have to cr...,"[-0.015282154, 0.0018193707, -0.020907281, 0.0...",0.152157
6,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Faena is over-rated. \n\nIt would be a 3 star ...,Overrated. There are better luxury options on ...,title overrated there are better luxury option...,"[-0.023257997, -0.015723582, 0.0051887627, 0.0...",0.174706
7,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,The service at the Faena is exceptional. From ...,5 star service ALL around,title 5 star service all around content the se...,"[0.027946766, 0.01693837, -0.006997557, 0.0678...",0.113305
8,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,My girlfriend and I had the dreamiest stay! Ev...,The perfect l

In [ ]:
df_with_embedding.shape

(10, 30)

In [ ]:
# cosine_similarity(corpus_embeddings, query_embedding)

#Entire Data

In [ ]:
startTime = time.time()

# Create a column named 'embedding', where the 'combined' column is turned to embeddings by the model.
df_combined["embedding"] = df_combined.combined.apply(lambda x: embedder.encode(x))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))


Execution time in seconds: 41.30338644981384


In [ ]:
# Turn the dataframe you have just created to pickle file, for later use.
df_combined.to_pickle('/content/drive/MyDrive/Semantic_Search/entire_data.pkl')

##Embedding upload - new starting point


In [ ]:
!pip install openai==0.27.7
!pip install gradio
!pip install -U sentence-transformers

In [ ]:
# drive.mount() loads the contents from your Google Drive.

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import pandas as pd
import time
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
import nltk
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
# import tiktoken
from openai.embeddings_utils import get_embedding, cosine_similarity

In [ ]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch

embedder = SentenceTransformer('all-mpnet-base-v2')

# Use the GPU if available
if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")
else:
  print("GPU Found!")
  embedder =  embedder.to('cuda')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


GPU Found!


In [ ]:
# Read the pickle file you entered earlier.
import pandas as pd
df = pd.read_pickle('/content/drive/MyDrive/Semantic_Search/entire_data.pkl') #to load 123.pkl back to the dataframe df

In [ ]:
df.shape

(2511, 29)

In [ ]:
df.head()

,id,type,name,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,longitude,webUrl,website,rankingString,rankingDenominator,numberOfReviews,review,title,combined,embedding
0,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Hands down my absolute favorite hotel in South...,My favorite hotel in Miami--Petar is the best!,title my favorite hotel in miamipetar is the b...,"[-0.03406972, 0.00075161044, -0.006820292, -0...."
1,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,There are not enough words to explain the sump...,Exquisite Stay,title exquisite stay content there are not eno...,"[-0.003170278, 0.0046354914, -0.034291837, 0.0..."
2,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I went for the Burlesque show which was absolu...,Burlesqu Show,title burlesqu show content i went for the bur...,"[-0.04177331, 0.04754442, -0.0032742221, 0.004..."
3,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,Dana at Gitano made the night magical! He make...,Dana at Gitano is a rockstar!,title dana at gitano is a rockstar content dan...,"[0.038456447, 0.073768765, -0.0072073163, 0.03..."
4,7787044,HOTEL,Faena Miami Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],5,$$$$,"$729 - $1,426",hotel,4.5,...,-80.12364,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.faena.com/miami-beach,#5 of 235 hotels in Miami Beach,235,2123,I would like to give our very very heartfelt t...,Perfect Place,title perfect place content i would like to gi...,"[-0.0047579487, 0.051271785, -0.047507547, 0.0..."


In [ ]:
# search through the reviews for a specific product
def search_reviews(df, query, n=5, pprint=True):

    # Embed your search query.
    query_embedding = embedder.encode(query,show_progress_bar=True)

    # As before, create a 'similarity' column, which shows the cosine similarity between the your query and the embedded combined contents.
    # REMEMBER!! YOU ARE USING THE SAME MODEL TO EMBED BOTH THE COMBINED CONTENTS AND YOUR QUERY.
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1))) #similarity against each doc

    # Now, sort the values bty similarity, and choose the most similar doc.
    results = (
        df.sort_values("similarity", ascending=False) # re-rank
        .head(n))

    return results

In [ ]:
query = 'A great view of Miami beach and close to Ocean Drive'

In [ ]:
results = search_reviews(df,query,15)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
results

,id,type,name,image,awards,rankingPosition,priceLevel,priceRange,category,rating,...,webUrl,website,rankingString,rankingDenominator,numberOfReviews,review,title,combined,embedding,similarity
716,217410,HOTEL,Hilton Grand Vacations Club McAlpin Ocean Plaz...,https://media-cdn.tripadvisor.com/media/photo-...,[],38,$$$$,$450,hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.hilton.com/en/hotels/miamsgv-hilto...,#38 of 235 hotels in Miami Beach,235,1257,"Location, location location. That is what McAl...",Great Miami Beach Location,title great miami beach location content locat...,"[-0.033547867, 0.017948722, 0.0027541576, 0.01...",[0.8234686]
233,15307694,HOTEL,Hampton Inn Miami Beach - Mid Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],4,$$$,$180 - $318,hotel,5.0,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.hilton.com/en/hotels/miacahx-hampt...,#4 of 235 hotels in Miami Beach,235,792,This hotel is very modern and in a great locat...,Best Miami Beach Location,title best miami beach location content this h...,"[-0.031054161, -0.04222299, -0.0051262034, 0.0...",[0.78161067]
2354,1449858,HOTEL,The Local House,https://media-cdn.tripadvisor.com/media/photo-...,[],46,$$$,$211 - $382,hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://localhouse.com,#46 of 235 hotels in Miami Beach,235,782,Very nice little boutique hotel right at the b...,Miami Beach Ocean Drive 400,title miami beach ocean drive 400 content very...,"[-0.001889081, 0.044548485, 0.0075611817, 0.03...",[0.76844674]
1378,240165,HOTEL,Sagamore Hotel,https://media-cdn.tripadvisor.com/media/photo-...,[],107,$$$$,$243 - $447,hotel,4.0,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.sagamoresouthbeach.com/,#107 of 235 hotels in Miami Beach,235,2933,"Great location, you can walk from the pool to ...",Miami Sagamore,title miami sagamore content great location yo...,"[-0.05113576, 0.042851184, -0.015707424, 0.004...",[0.7656931]
2209,558250,HOTEL,"MB Hotel, Trademark Collection by Wyndham",https://media-cdn.tripadvisor.com/media/photo-...,[],157,$$$,$165 - $265,hotel,3.0,...,https://www.tripadvisor.com/Hotel_Review-g3443...,http://www.wyndhamhotels.com/hotel/48182,#157 of 235 hotels in Miami Beach,235,628,"Close to the beach for sunrise viewing, lovely...",Small and close to the beach,title small and close to the beach content clo...,"[-0.03469386, 0.0023410239, -0.0067653167, 0.0...",[0.7560585]
393,2008516,HOTEL,"Marriott Vacation Club Pulse, South Beach",https://media-cdn.tripadvisor.com/media/photo-...,[],49,$$$,$190 - $329,hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.marriott.com/en-us/hotels/miamv-ma...,#49 of 235 hotels in Miami Beach,235,1864,South Beach Pulse surpassed all expectations w...,Best Vacation Property in Miami,title best vacation property in miami content ...,"[-0.06324199, -0.0018072735, 0.0002525109, -0....",[0.7553634]
2505,23526747,HOTEL,Urbanica Fifth,https://media-cdn.tripadvisor.com/media/photo-...,[],11,$$$,$177 - $329,hotel,4.5,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://urbanicahotels.com/the-fifth/,#11 of 235 hotels in Miami Beach,235,155,"This hotel is in a great location, just a shor...",Great Location,title great location content this hotel is in ...,"[-0.018627916, -0.04268277, 0.0054982365, 0.01...",[0.74287885]
257,15307694,HOTEL,Hampton Inn Miami Beach - Mid Beach,https://media-cdn.tripadvisor.com/media/photo-...,[],4,$$$,$180 - $318,hotel,5.0,...,https://www.tripadvisor.com/Hotel_Review-g3443...,https://www.hilton.com/en/hotels/miacahx-hampt...,#4 of 235 hotels in Miami Beach,235,792,"If you’re looking for value, this is the place...",Miami Beach trip,title miami beach trip content if youre lookin...,"[-0.023794277, 0.007490118, -0.014957584, 0.00...",[0.7422469]
2191,1188680,HOTEL,The Fritz Hotel,https://media-cdn.tripadvisor.com/media/photo-...,[],56,$$$,$168 - $293,hotel,4.5,...,https://www.tripa

Take all the reviews which are closest to the query, and groupby the hotel name

In [ ]:
def search(query):
  # Define a number of results to return, in this case, return only the first 15 results ranked by similarity.
  n = 15

  # Embed the query.
  query_embedding = embedder.encode(query)

  # Generate the similarity column, based on your query.
  df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1)))

  # Calculate the top 'n' most similar results by similarity.
  results = (
      df.sort_values("similarity", ascending=False)
      .head(n))

  resultlist = []

  # Display them in a very concise and ordered manner.
  hlist = []
  for r in results.index:
      if results.name[r] not in hlist:
          smalldf = results.loc[results.name == results.name[r]]
          if smalldf.shape[1] > 3:
            smalldf = smalldf[:3]

          resultlist.append(
          {
            "name":results.name[r],
            "score": smalldf.similarity[r][0],
            "rating": smalldf.rating.max(),
            "relevant_reviews": [ smalldf.review[s] for s in smalldf.index]
          })
          hlist.append(results.name[r])
  return resultlist




In [ ]:
search('A great view of Miami beach and close to Ocean Drive')

[{'name': 'Hilton Grand Vacations Club McAlpin Ocean Plaza Miami',
  'score': 0.8234686,
  'rating': 4.5,
  'relevant_reviews': ['Location, location location. That is what McAlpin Ocean Resort has going for it. It’s at the north end of Ocean Drive, which is quieter and classier than a few blocks south. This portion of Ocean Drive is, thankfully, still closed to auto traffic. You can sit out on the front porch area of either building and watch the world go by. Arlene, the assistant manager, was most helpful during our stay. She made the difference. The rooms are smaller than most HGV properties, but adequate. Ask for a higher floor since rooms on the 1st and 2nd floor tend to be darker since the next door properties are quit close. Both buildings have a nice jacuzzi on the roof with lounge chairs. Free coffee and tea all day. The beach is directly across the street. Lots of restaurants are close by.']},
 {'name': 'Hampton Inn Miami Beach - Mid Beach',
  'score': 0.78161067,
  'rating': 

In [ ]:
from google.colab import userdata
import openai
openai.api_key = userdata.get('OPENAI_API_KEY')


SecretNotFoundError: Secret OPENAI_API_KEY does not exist.

In [ ]:
def generate_answer(query):
    prompt = f"""
    Based on the following query from a user, please generate a detailed answer based on the context
    focusing on which is the top three hotel based on the query. You should respond as you are a travel agent and are conversing with the
    user in a nice cordial way. Always address the user as Arnav. Make sure you infrom the user why this is a good answer. The answer should be a paragph
    remove the special characters and (/n ) , make the output clean and concise.
    Answer like you are a poet and physicist.

    ###########
    query:
    "{query}"

    ########

    context:"
    "{search(query)}"
    #####

    Return in Markdown format with each hotel highlighted.
    """

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        max_tokens=1500,
        n=1,
        stop=None,
        temperature=0.2, #higher temperature means more creative or more hallucination
        messages = messages


    )

    # Extract the generated response from the API response
    generated_text = response.choices[0].message['content'].strip()

    return generated_text

# # Example usage
# query = "What are the best amenities offered by Hotel XYZ?"
# response = generate_hotel_response(query)
# print(response)

    #return response.choices[0].message.content.strip()

In [ ]:
txt=generate_answer('close to Louvre and great food nearby but not too expensive')

In [ ]:
# prompt

In [ ]:
import markdown
from IPython.display import display, HTML

def render_markdown(md_text):
    # Convert Markdown to HTML
    html = markdown.markdown(md_text)
    # Display the HTML
    display(HTML(html))

In [ ]:
render_markdown(txt)

#Building the API

In [ ]:

import gradio as gr

# def search(query):
#   n = 15
#   query_embedding = embedder.encode(query)
#   df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1)))

#   results = (
#       df.sort_values("similarity", ascending=False)
#       .head(n))

#   resultlist = []

#   hlist = []
#   for r in results.index:
#       if results.name[r] not in hlist:
#           smalldf = results.loc[results.name == results.name[r]]
#           smallarr = smalldf.similarity[r].max()
#           sm =smalldf.rating[r].mean()

#           if smalldf.shape[1] > 3:
#             smalldf = smalldf[:3]

#           resultlist.append(
#           {
#             "name":results.name[r],
#             "description":results.description[r],
#             "relevance score": smallarr.tolist(),
#             "rating": sm.tolist(),
#             "relevant_reviews": [ smalldf.text[s] for s in smalldf.index]
#           })
#           hlist.append(results.name[r])
#   return resultlist

def greet(query):
    bm25 = generate_answer(query)
    return bm25

# Use the gradio library to display a user interface for your user to interact with.
demo = gr.Interface(fn=greet, inputs="text", outputs="text")

# Launch the user demo - This can be done directly in your colab notebook. On your local notebook, you can also give a personalized localhost:port address.
demo.launch(share=True,debug=False)
# COPY THE URL THAT APPEARS BELOW

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://07955ce1b7b0132dee.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install --upgrade gradio

In [ ]:
# Import the Client class from the gradio_client module.
from gradio_client import Client

# Create an instance of the Client class. The URL provided should point to a live Gradio app.

# PASTE THE URL FROM ABOVE HERE
client = Client("https://7142ca2aabef402984.gradio.live/")

# Use the 'predict' method of the Client instance to send a request to the Gradio app.
# The string "Hotel near the Eiffel Tower!" is passed to the 'query' textbox component of the app.
# 'api_name' specifies the endpoint ('/predict') that the Gradio interface exposes for processing this input.
result = client.predict(
				"Hotel near the Eiffel Tower!",	# str in 'query' Textbox component
				api_name="/predict"
)

# Print the result returned from the Gradio app. This output depends on how the Gradio app processes the input.
print(result)



Loaded as API: https://7142ca2aabef402984.gradio.live/ ✔


ValueError: Could not fetch config for https://7142ca2aabef402984.gradio.live/

In [ ]:
result